# Options

In [81]:
process_full_dataset = False

# Credit Risk Resampling Techniques

In [82]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [84]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [85]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [86]:
# Take a slice to speed up processing
# if not process_full_dataset:
#     df["loan_status"].unique()
#     low_risk_subset = df.copy().loc[df["loan_status"] == "low_risk"][:100]
#     high_risk_subset = df.copy().loc[df["loan_status"] == "high_risk"][:100]
#     df = low_risk_subset.append(high_risk_subset).head()

df2 = df.copy()
# df2["loan_status"].unique()
low_risk_subset = df2.loc[df2["loan_status"] == "low_risk"][:20]
high_risk_subset = df2.loc[df2["loan_status"] == "high_risk"][:20]
df = low_risk_subset.append(high_risk_subset)


In [87]:
# df = low_risk_subset.append(high_risk_subset).head()
[low_risk_subset.shape,
high_risk_subset.shape,
df.shape]
# df
# low_risk_subset.shape
# high_risk_subset.shape

[(20, 86), (20, 86), (40, 86)]

In [88]:
low_risk_subset.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [89]:
high_risk_subset.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
4261,4400.0,0.2000,163.52,RENT,41000.0,Not Verified,Mar-2019,high_risk,n,31.82,...,100.0,0.0,0.0,0.0,38014.0,27390.0,5400.0,25314.0,N,N
4464,5000.0,0.2880,208.99,MORTGAGE,80000.0,Source Verified,Mar-2019,high_risk,n,16.88,...,100.0,80.0,0.0,0.0,551300.0,113366.0,10700.0,8600.0,N,N
4539,5400.0,0.0819,169.70,RENT,58000.0,Not Verified,Mar-2019,high_risk,n,36.50,...,100.0,20.0,0.0,0.0,374414.0,476443.0,29200.0,317214.0,N,N
4930,11000.0,0.1474,379.92,MORTGAGE,80000.0,Source Verified,Mar-2019,high_risk,n,19.82,...,71.4,42.9,0.0,0.0,248484.0,52308.0,16100.0,61275.0,N,N
4979,1000.0,0.1797,36.14,RENT,21456.0,Not Verified,Mar-2019,high_risk,n,16.72,...,94.1,0.0,0.0,0.0,47375.0,19150.0,28200.0,19175.0,N,N


In [90]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our target
y = df["loan_status"]

In [91]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.00000,40.000000,...,40.000000,40.0,40.0,40.000000,40.000000,40.0,40.000000,40.000000,40.0,40.0
mean,13962.500000,0.150358,425.152250,75308.300000,23.982000,0.125000,0.550000,12.450000,0.15000,17563.700000,...,0.200000,1.0,1.0,0.125000,0.875000,1.0,0.900000,0.100000,1.0,1.0
std,7662.519084,0.049325,247.479211,52193.144939,13.097658,0.334932,0.749359,7.362239,0.36162,24638.649639,...,0.405096,0.0,0.0,0.334932,0.334932,0.0,0.303822,0.303822,0.0,0.0
min,1000.000000,0.064600,36.140000,21456.000000,4.830000,0.000000,0.000000,4.000000,0.00000,1129.000000,...,0.000000,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.0
25%,10000.000000,0.113650,305.840000,51828.750000,16.537500,0.000000,0.000000,8.000000,0.00000,6326.000000,...,0.000000,1.0,1.0,0.000000,1.000000,1.0,1.000000,0.000000,1.0,1.0
50%,14000.000000,0.155700,383.760000,64147.500000,19.915000,0.000000,0.000000,10.000000,0.00000,9956.500000,...,0.000000,1.0,1.0,0.000000,1.000000,1.0,1.000000,0.000000,1.0,1.0
75%,17175.000000,0.179700,518.657500,90000.000000,30.675000,0.000000,1.000000,14.250000,0.00000,18089.000000,...,0.000000,1.0,1.0,0.000000,1.000000,1.0,1.000000,0.000000,1.0,1.0
max,40000.000000,0.288000,1537.990000,350000.000000,62.800000,1.000000,2.000000,42.000000,1.00000,116737.000000,...,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0


In [92]:
# Check the balance of our target values
y.reset_index()['loan_status'].value_counts()

high_risk    20
low_risk     20
Name: loan_status, dtype: int64

In [93]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1)
X_train.shape


(30, 91)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [94]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [95]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scalar = scaler.fit(X_train)


In [96]:
# Scale the training and testing data
X_train_scaled = X_scalar.transform(X_train)
X_test_scaled = X_scalar.transform(X_test)


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [97]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)


Counter({'high_risk': 17, 'low_risk': 17})

In [98]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [99]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)


0.8571428571428572

In [100]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[5, 2],
       [0, 3]], dtype=int64)

In [101]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

# Precision (pre), Recall (rec), Accuracy (f1), Geometric (geo)
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.71      1.00      0.83      0.85      0.69         7
   low_risk       0.60      1.00      0.71      0.75      0.85      0.73         3

avg / total       0.88      0.80      0.91      0.81      0.85      0.71        10



### SMOTE Oversampling

In [102]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
from collections import Counter

Counter(y_resampled)


Counter({'high_risk': 17, 'low_risk': 17})

In [103]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [104]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)


0.8571428571428572

In [105]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)


array([[5, 2],
       [0, 3]], dtype=int64)

In [106]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.71      1.00      0.83      0.85      0.69         7
   low_risk       0.60      1.00      0.71      0.75      0.85      0.73         3

avg / total       0.88      0.80      0.91      0.81      0.85      0.71        10



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [107]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1, sampling_strategy=1.0)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

from collections import Counter

Counter(y_resampled)


Counter({'high_risk': 13, 'low_risk': 13})

In [108]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [109]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)


0.8571428571428572

In [110]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)


array([[5, 2],
       [0, 3]], dtype=int64)

In [111]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.71      1.00      0.83      0.85      0.69         7
   low_risk       0.60      1.00      0.71      0.75      0.85      0.73         3

avg / total       0.88      0.80      0.91      0.81      0.85      0.71        10



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [112]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)


Counter({'high_risk': 6, 'low_risk': 3})

In [113]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [114]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)


0.8571428571428572

In [115]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)


array([[7, 0],
       [2, 1]], dtype=int64)

In [116]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.78      1.00      0.33      0.88      0.58      0.36         7
   low_risk       1.00      0.33      1.00      0.50      0.58      0.31         3

avg / total       0.84      0.80      0.53      0.76      0.58      0.34        10

